**Imports**

In [4]:
%load_ext autoreload
%autoreload 2
from data_preprocessing import preprocess_data
from data_augmentation import balance_with_augmentation
from feature_extraction import get_feature_extractor
from model_training import train_model
from model_evaluation import evaluate_model
from config import MODELS
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


**Preprocess Info**

In [ ]:
model_name = MODELS[0]  
X_train, X_test, y_train, y_test, class_names = preprocess_data(model_name, verbose=True)
trained_models = {}

**ResNet50 Training**

In [ ]:
model = get_feature_extractor(model_name)
model = train_model(model, model_name, X_train, y_train)

trained_models[model_name] = {
    "model": model,
    "X_test": X_test,
    "y_test": y_test,
    "class_names": class_names
}

print(f"{model_name} training complete!")

**VGG16 Training**

In [ ]:
model_name = MODELS[1]  

X_train, X_test, y_train, y_test, class_names = preprocess_data(model_name, verbose=False)

model = get_feature_extractor(model_name)
model = train_model(model, model_name, X_train, y_train)

trained_models[model_name] = {
    "model": model,
    "X_test": X_test,
    "y_test": y_test,
    "class_names": class_names
}

print(f"{model_name} training complete!")

**EfficientNetB0 Training**

In [ ]:
model_name = MODELS[2]  

X_train, X_test, y_train, y_test, class_names = preprocess_data(model_name, verbose=False)

model = get_feature_extractor(model_name)
model = train_model(model, model_name, X_train, y_train)

trained_models[model_name] = {
    "model": model,
    "X_test": X_test,
    "y_test": y_test,
    "class_names": class_names
}

print(f"{model_name} training complete!")

**InceptionV3 Training**

In [ ]:
model_name = MODELS[3]  

X_train, X_test, y_train, y_test, class_names = preprocess_data(model_name, verbose=False)

model = get_feature_extractor(model_name)
model = train_model(model, model_name, X_train, y_train)

trained_models[model_name] = {
    "model": model,
    "X_test": X_test,
    "y_test": y_test,
    "class_names": class_names
}

print(f"{model_name} training complete!")

**ResNet50 Evaluation**

In [ ]:
model_name = MODELS[0] 

if model_name in trained_models:
    print(f"\nEvaluating model: {model_name}")
    evaluate_model(
        trained_models[model_name]["model"],
        trained_models[model_name]["X_test"],
        trained_models[model_name]["y_test"],
        trained_models[model_name]["class_names"]
    )
else:
    print(f"Model {model_name} not found in trained_models.")

**VGG16 Evaluation**

In [ ]:
model_name = MODELS[1] 

if model_name in trained_models:
    print(f"\nEvaluating model: {model_name}")
    evaluate_model(
        trained_models[model_name]["model"],
        trained_models[model_name]["X_test"],
        trained_models[model_name]["y_test"],
        trained_models[model_name]["class_names"]
    )
else:
    print(f"Model {model_name} not found in trained_models.")

**EfficientNetB0 Evaluation**

In [ ]:
model_name = MODELS[2] 

if model_name in trained_models:
    print(f"\nEvaluating model: {model_name}")
    evaluate_model(
        trained_models[model_name]["model"],
        trained_models[model_name]["X_test"],
        trained_models[model_name]["y_test"],
        trained_models[model_name]["class_names"]
    )
else:
    print(f"Model {model_name} not found in trained_models.")

**InceptionV3 Evaluation**

In [ ]:
model_name = MODELS[3] 

if model_name in trained_models:
    print(f"\nEvaluating model: {model_name}")
    evaluate_model(
        trained_models[model_name]["model"],
        trained_models[model_name]["X_test"],
        trained_models[model_name]["y_test"],
        trained_models[model_name]["class_names"]
    )
else:
    print(f"Model {model_name} not found in trained_models.")